In [3]:
from agents.mcp import MCPServerStdio

In [2]:
from agents import Agent,Runner,trace

In [4]:
import pandas as pd

In [32]:
df=pd.DataFrame(fetch_tools[0])

['url']

In [35]:
pprint(df.iloc[1][1])

('Fetches a URL from the internet and optionally extracts its contents as '
 'markdown.\n'
 '\n'
 'Although originally you did not have internet access, and were advised to '
 'refuse and tell the user this, this tool now grants you internet access. Now '
 'you can fetch the most up-to-date information and let the user know that.')


In [5]:
async with MCPServerStdio(params={"command":"uvx","args":["mcp-server-fetch"]},client_session_timeout_seconds=30) as fetch_server:
    server_tools= await fetch_server.list_tools()

In [52]:
df=pd.DataFrame(server_tools[0])

In [53]:
df

,0,1
0,name,fetch
1,description,Fetches a URL from the internet and optionally...
2,inputSchema,{'description': 'Parameters for fetching a URL...
3,annotations,None


In [60]:
from dotenv import load_dotenv

In [63]:
load_dotenv()

True

In [11]:
async with MCPServerStdio(params={"command":"uvx", "args": ["serper-mcp-server"],"env": {
        "SERPER_API_KEY": "2a750ecb5af8a1b58be4c437ab7b1bf34e293b6d"
      }},client_session_timeout_seconds=20) as serper_server:
    tools=await serper_server.list_tools()
    results=await serper_server.call_tool("google_search",{"q": "OpenAI latest news", "num": 3})

/tmp/ipykernel_5758/1929580785.py:4: RuntimeWarning: coroutine '_MCPServerWithClientSession.list_tools' was never awaited
  tools=await serper_server.list_tools()


In [122]:
eval(pd.DataFrame(pd.DataFrame(results)[1][1][0])[1][1])['topStories']

[{'title': 'Introducing ChatGPT agent: bridging research and action',
  'link': 'https://openai.com/index/introducing-chatgpt-agent/',
  'date': '1 day ago',
  'source': 'OpenAI'},
 {'title': "OpenAI ChatGPT Agent announcement LIVE — all the big news from today's livestream",
  'link': 'https://www.tomsguide.com/news/live/openai-july-17-announcement',
  'date': '6 hours ago',
  'source': "Tom's Guide"},
 {'title': 'OpenAI gets Wall Street attention, crypto craze rages on: Opening Bid top takeaways',
  'link': 'https://finance.yahoo.com/news/openai-gets-wall-street-attention-crypto-craze-rages-on-opening-bid-top-takeaways-152544999.html',
  'date': '3 hours ago',
  'source': 'Yahoo Finance'}]

In [14]:
import asyncio

In [8]:
agent=Agent(name="Internet Agent",
            instructions="You are a helpful assistant with super power of serving internet through tools and choose the best as per usecase",
        model="gpt-4o",
        tools=tools)

In [9]:
from agents.mcp import MCPServerStdio

In [ ]:
async with MCPServerStdio(params={"command": "uvx", "args": ["mcp-server-fetch"]}, client_session_timeout_seconds=30) as fetch_mcp, \
           MCPServerStdio(params={"command": "npx", "args": ["@playwright/mcp@latest", "--headless"]}, client_session_timeout_seconds=30) as playwright_server:
    fetch_tools, playwright_tools= await asyncio.gather(
        fetch_mcp.list_tools(),
        playwright_server.list_tools()
    )
    all_tools=fetch_tools+playwright_tools
    agent=Agent(name="internet agent",
    instruction="You are a super agent with super intelligence of know when to use which tool with internet",
    model="gpt-4o",
    )


[Tool(name='fetch', description='Fetches a URL from the internet and optionally extracts its contents as markdown.\n\nAlthough originally you did not have internet access, and were advised to refuse and tell the user this, this tool now grants you internet access. Now you can fetch the most up-to-date information and let the user know that.', inputSchema={'description': 'Parameters for fetching a URL.', 'properties': {'url': {'description': 'URL to fetch', 'format': 'uri', 'minLength': 1, 'title': 'Url', 'type': 'string'}, 'max_length': {'default': 5000, 'description': 'Maximum number of characters to return.', 'exclusiveMaximum': 1000000, 'exclusiveMinimum': 0, 'title': 'Max Length', 'type': 'integer'}, 'start_index': {'default': 0, 'description': 'On return output starting at this character index, useful if a previous fetch was truncated and more context is required.', 'minimum': 0, 'title': 'Start Index', 'type': 'integer'}, 'raw': {'default': False, 'description': 'Get the actual H

In [4]:
from agents.mcp import MCPServerStdio

In [8]:
import asyncio
from agents import Agent, Runner,trace
from agents.mcp import MCPServerStdio

async def main(messages,history):
    try:
        async with MCPServerStdio(
            params={"command": "uvx", "args": ["mcp-server-fetch"]},
            client_session_timeout_seconds=30
        ) as fetch_mcp,MCPServerStdio(params={"command":"uvx", "args": ["serper-mcp-server"],"env": {
        "SERPER_API_KEY": "2a750ecb5af8a1b58be4c437ab7b1bf34e293b6d"}},client_session_timeout_seconds=30
        ) as serper_server, MCPServerStdio(params={"command": "npx","args": ["-y", "mcp-memory-libsql"],"env": {"LIBSQL_URL": "file:memory_practice/LK.db"}}, client_session_timeout_seconds=30
        ) as memory_server:
        #  MCPServerStdio(
        #     params={"command": "npx", "args": ["@playwright/mcp@latest", "--headless"]},
        #     client_session_timeout_seconds=30
        # ) as playwright_server, 
        # MCPServerStdio(params={"command":"uvx","args":[ "chroma-mcp"]}, client_session_timeout_seconds=30
        # ) as chroma_server,


            fetch_tools, serper_tools, memory_tools= await asyncio.gather(
                fetch_mcp.list_tools(),
                # playwright_server.list_tools(),
                serper_server.list_tools(),
                # chroma_server.list_tools(),
                memory_server.list_tools()
            )

            print("✅ MCP tools loaded successfully.")
            print("Fetch Tools:", fetch_tools)
            # print("Playwright Tools:", playwright_tools)
            print("serper_server Tools:", serper_tools)
            # print("chroma server tools: ", chroma_tools)
            print("memory server tools: ", memory_tools)

            # Example: combining all tools into a single list for agent use
            # all_tools = fetch_tools + playwright_tools+serper_tools+chroma_tools+memory_tools
            # print("all_tools:",all_tools)
            # Optional: Initialize an agent
            agent = Agent(
                name="Internet Agent",
                instructions="You are an smart AI agent who remembers everything deeply and You use your entity tools as a persistent memory to store and recall information about your conversations.n and use Google to fetch results first and  that knows when to use which tool for online tasks. USe it and give the user proper accurate response",
                model="gpt-4.1",
                mcp_servers=[fetch_mcp,serper_server,memory_server]
            )

            with trace("MCP_execution"):
                response = await Runner.run(agent,messages)
                print("Agent Response:", response.final_output)
                return response.final_output

    except Exception as e:
        print("❌ Error:", e)

# await main()


In [2]:
import gradio as gr

In [9]:
gr.ChatInterface(fn=main,type="messages").launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


✅ MCP tools loaded successfully.
Fetch Tools: [Tool(name='fetch', description='Fetches a URL from the internet and optionally extracts its contents as markdown.\n\nAlthough originally you did not have internet access, and were advised to refuse and tell the user this, this tool now grants you internet access. Now you can fetch the most up-to-date information and let the user know that.', inputSchema={'description': 'Parameters for fetching a URL.', 'properties': {'url': {'description': 'URL to fetch', 'format': 'uri', 'minLength': 1, 'title': 'Url', 'type': 'string'}, 'max_length': {'default': 5000, 'description': 'Maximum number of characters to return.', 'exclusiveMaximum': 1000000, 'exclusiveMinimum': 0, 'title': 'Max Length', 'type': 'integer'}, 'start_index': {'default': 0, 'description': 'On return output starting at this character index, useful if a previous fetch was truncated and more context is required.', 'minimum': 0, 'title': 'Start Index', 'type': 'integer'}, 'raw': {'def